In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-70b-8192")

## Runnable PassThrough

* It does not do anything to the input data.
* Let's see it in a very simple example: chain with just RunnablePassThrough() will output the original input without any modifications.

In [4]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [5]:
chain.invoke("Lokesh")

'Lokesh'

## Runnable Lambda

* To use custom function inside a LCEL chain, we need to wrap it up with Runnable Lambda.
* Let's define a very simple function to create Russian lastnames:

In [6]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [7]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [8]:
chain.invoke("Lokesh")

'Lokeshovich'

## Runnable Parallel

* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    1. operation_a will use RunnablePassthrough.
    2. operation_b will use RunnableLambda with a russian_lastname function.

In [9]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [10]:
chain.invoke("Lokesh")

{'operation_a': 'Lokesh', 'operation_b': 'Lokeshovich'}

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [12]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [13]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [15]:
chain.invoke({
    "name1": "Messi",
    "name": "Abram",
})

'Here\'s one:\n\nRoman Abramovich, the billionaire owner of Chelsea FC, has a rather unusual hobby - he\'s an avid collector of frozen mammoths! Yes, you read that right. In 2012, he purchased a 39,000-year-old frozen mammoth carcass for a staggering $1.5 million at an auction in New York. The mammoth, known as "Yuka," is one of the most well-preserved mammoth carcasses ever found, with skin, hair, and even its brain still intact. Abramovich\'s fascination with mammoths is said to be driven by his interest in science and history. Who knew that a Russian oligarch and a frozen mammoth would make for an intriguing combination?'

### Let's see a more advanced use of Runnable Parallel

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq 

In [18]:
vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content related to Data Science, AI, ML, NLP, DL, CV, Python Programming etc. in English"], 
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatGroq(model="llama3-70b-8192")

retriever_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retriever_chain.invoke("What is dswithbappy?")

'According to the context, dswithbappy is a source that focuses on providing content related to various topics such as Data Science, AI, ML, NLP, DL, CV, and Python Programming, all in English.'

### Alternate way to use the runnable parallel

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq 

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content related to Data Science, AI, ML, NLP, DL, CV, Python Programming etc. in English"], 
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatGroq(model="llama3-70b-8192")

#Whenever you see Curly braces{} inside chain, then it will be act as RunnableParallel
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is dswithbappy?", "language": "Hindi"})

'ड्सविथबैप्पी डेटा साइंस, एआई, एमएल, एनएलपी, डीएल, सीवी, पायथन प्रोग्रामिंग आदि से संबंधित सामग्री प्रदान करने पर केंद्रित है।'

In [21]:
chain.invoke({"question": "What is dswithbappy?", "language": "Marathi"})

'डीसविथबप्पी हे डेटा सायन्स, एआय, एमएल, एनएलपी, डीएल, सीव्ही, पायथन प्रोग्रामिंग इत्यादी विषयांवर इंग्रजी भाषेत सामग्री पुरवठा करणारे मंच आहे.'

In [22]:
chain.invoke({"question": "What is dswithbappy?", "language": "Spanish"})

'Según el contexto, dswithbappy se centra en proporcionar contenido relacionado con Ciencia de Datos, IA, ML, NLP, DL, CV y programación en Python, por lo que se puede inferir que dswithbappy es un sitio web, blog o plataforma de contenido en línea que se enfoca en temas de tecnología y programación.'